Interface to KMyMoney saved files
===============
You must save your data as SQL (or you can export your current data to an sql file)

In [ ]:
import pandas as pd
from IPython.display import HTML
import matplotlib.pyplot as plt

class ACCOUNT_TYPE:
    INCOME = '12'
    EXPENSE = '13'
    STOCK = '15'
    EQUITY = '16'
    
DEFAULT_CURRENCY = 'EUR'   # ??? should be computed from FileInfo.baseCurrency
    

class KMyMoney:
    def __init__(self, filename):
        self.sqlite = 'sqlite:///{}'.format(filename)
        
    def _qaccount_name(self):
        """
        Return the Common Table Expression to compute the fully qualified account
        names, including the parent acconuts
        """
        return """qAccountName(accountId, accountType, name, accountName, currencyId) AS (
            SELECT kmmAccounts.id, kmmAccounts.accountType,
                   kmmAccounts.accountName, kmmAccounts.accountName, kmmAccounts.currencyId
               FROM kmmAccounts
               WHERE kmmAccounts.parentId IS NULL
            UNION ALL
            SELECT kmmAccounts.id, kmmAccounts.accountType,
                   qAccountName.name || ':' || kmmAccounts.accountName,
                   kmmAccounts.accountName, kmmAccounts.currencyId
               FROM qAccountName, kmmAccounts
               WHERE kmmAccounts.parentId = qAccountName.accountId
        )
        """
    
    def _to_float(self, fieldname):
        """
        Converts a "n/m" value as stored by kMyMoney to a float.
        The `*Formatted` field in the database seem wrong sometimes.
        """
        return (
            f"1.0 * substr({fieldname}, 0, instr({fieldname}, '/'))"
            f"/substr({fieldname}, instr({fieldname}, '/') + 1)"
        )
    
    def _splits_and_fees(self, currency=DEFAULT_CURRENCY):
        """
        return the Common Table Expression to compute the list of
        all transactions include their fees. This works for both checking
        transactions (which have no fee) and transactions on investments.
        
        This function also computes quantities, prices and values from
        the "n/m" fields in the database, since the equivalent sharesFormatted,
        priceFormatted and valueFormatted seem to be wrong sometimes.

        NOTE:
        Only fees in the given currency are taken into account (so if you
        have a mix of currencies for those fees, some will be ignored).        
        """
        return f"""splits_and_fees AS (
       SELECT 
          s.transactionId,
          s.splitId,
          s.accountId,
          s.action,
          {self._to_float('s.shares')} as quantity,
          {self._to_float('s.price')} as price,
          {self._to_float('s.value')} as value,
          s.postDate,          
          SUM({self._to_float('t.value')}) as fees
       FROM 
          kmmSplits s
          JOIN kmmAccounts ON (s.accountId = kmmAccounts.id)          
          LEFT JOIN 
            (kmmSplits
             JOIN kmmAccounts ta 
             ON (
                kmmSplits.accountId = ta.id 
                AND ta.accountType = 13          --  fees are computed on Expense accounts
                AND ta.currencyId = "{currency}"
             )
         ) t
         ON (s.transactionId = t.transactionId          
             AND s.splitId != t.splitId
             
             --   Fees only apply to Stock accounts
             AND kmmAccounts.accountType = {ACCOUNT_TYPE.STOCK}
             )
       GROUP BY 1,2,3,4,5,6,7,8
    )
         """
    
    def _price_history(
        self,
        currency,
    ):
        """
        Return the common table expression to compute the price history for
        stocks, including the full time range that this price
        applies.
        """
        return f"""price_history AS (
       SELECT kmmPrices.*, 
          {self._to_float('price')} as computedPrice,
          (
             SELECT COALESCE(MIN(priceDate), '9000-01-01') 
             FROM kmmPrices m
             WHERE kmmPrices.fromId = m.fromId
               AND m.priceDate > kmmPrices.priceDate
          ) as maxDate
       FROM kmmPrices
       WHERE kmmPrices.toId = "{currency}"
      )
        """
    
    def _test_accounts(self, tablename="kmmSplits", accounts=None):
        """
        Restrict a query to a specific set of accounts.
        :param accounts:
           either None (all accounts), a string for the name of a single account,
           or a list of account ids.
        """    
        if isinstance(accounts, (list, tuple)):
            return (
                f" AND {tablename}.accountid in (%s)"
                % ",".join("'%s'" % a for a in accounts)  # ??? unsafe
            )
        elif accounts:
            return f" AND {tablename}.accountid = '{accounts}'"
        else:
            return ""

    def networth(
        self,
        accounts=None,
        currency=DEFAULT_CURRENCY,
        by_year=False,
        mindate=None,
        maxdate=None,  # "2020-12-31"  (end of period)
        with_total=True,
    ):
        """
        Compute the networth for all accounts at the end of each month or year
        in the given date range. The result is a pivot table.
        """
        
        if by_year:
            # this must be the first date in the file with transactions, since
            # balances are computed by adding all transactions from the beginning
            # of times.
            # ??? should be computed
            # It should be set to end of the periods
            earliest_date = "2009-12-31"
            inc = "+1 YEARS"
            formatted = "%Y"
            
        else:
            earliest_date = "2009-01-01"
            inc = "+1 MONTHS"
            formatted = "%Y-%m"
        
        p = pd.read_sql(
            f"""
        WITH RECURSIVE 

        --  Generate the set of dates for which we want balances
        all_dates(d) AS (
            SELECT "{earliest_date}"
            UNION ALL
            SELECT DATE(d, "{inc}") from all_dates where DATE(d, "{inc}") <= "{maxdate}"
        ),

        --  We will need to compute the prices at the end of each period
        {self._price_history(currency)},

        --  The full list of accounts, including fully qualified names
        {self._qaccount_name()},

        --  For each account, the balanceShare after the last transaction
        --  during each year. If there was no transaction, NULL is returned
        balances AS (
           SELECT DISTINCT
               strftime("{formatted}", s.postDate) as date,
               s.accountId,
               LAST_VALUE(s.balanceShares)
                  OVER (PARTITION BY strftime("{formatted}", s.postDate),
                                     s.accountId
                        ORDER BY s.postDate ASC
                        RANGE BETWEEN UNBOUNDED PRECEDING
                          AND UNBOUNDED FOLLOWING          
                  ) as balanceShares
           FROM 
              (SELECT
                  s.postDate,   --  ??? Could use LAST_VALUE, and a GROUP BY
                  s.accountId,
                   --  compute a running total of shares, per account
                  SUM({self._to_float('s.shares')})
                    OVER (PARTITION BY s.accountid 
                          ORDER BY s.postDate 
                          ROWS BETWEEN UNBOUNDED PRECEDING 
                             AND CURRENT ROW
                             --  AND UNBOUNDED FOLLOWING
                         ) as balanceShares           
               FROM kmmSplits s
              ) s   
        )

        SELECT all_dates.d as date,
           qAccountName.name as accountname,
           qAccountName.accountId,
           balances.balanceShares as balanceShares,
           coalesce(price_history.computedPrice, 1) as computedPrice
        FROM all_dates
          JOIN qAccountName
          LEFT JOIN balances
            ON (strftime("{formatted}", all_dates.d) = balances.date
                AND qAccountName.accountId = balances.accountId
               )
          LEFT JOIN price_history
            ON (price_history.fromId = qAccountName.currencyId
                AND all_dates.d >= price_history.priceDate
                AND all_dates.d < price_history.maxDate
            )

        WHERE qAccountName.accountType not in (:expense, :income, :equity)
           {self._test_accounts('qAccountName', accounts)}
            """,
            self.sqlite,
            params={
                "expense": ACCOUNT_TYPE.EXPENSE,
                "income": ACCOUNT_TYPE.INCOME,
                "equity": ACCOUNT_TYPE.EQUITY,
            }
        )
        
        pivot = pd.pivot_table(
            p,
            values=['balanceShares', 'computedPrice'],
            index=['accountname'],
            columns=['date'],
            dropna=False,
            margins=False,   # "total" rows and columns, but these would not include the result of ffill below
        )

        # ??? If all balances are None, the column disappears
        if 'balanceShares' not in pivot.columns:
            return None
                
        # Replace NaN with the last known value in the same row
        pivot = pivot.groupby(level=0).fillna(axis=1, method='ffill')

        # Compute the EUR balance, taking the number of shares of the last
        # transaction, with the price computed at the end of each period.
        p = pivot['balanceShares'] * pivot['computedPrice']

        # Only remove old columns after we have propagated values
        if mindate:
            p = p[[c for c in p.columns if c >= mindate]]

        if with_total:
            p =  pd.concat([
                p,
                # pivot.sum(level=0).assign(accountname='Subtotal')
                #   .set_index('accountname', append=True),
                p.sum().to_frame().T
                   .assign(accountname='Total')
                   .set_index(['accountname'])
            ]).sort_index()
        
        return p
    
    # See also https://github.com/quantopian/qgrid
    # simple code copied from  https://github.com/pandas-dev/pandas/issues/29072

    def disp(self, p, height=300):
        """
        Display a pivot table so that the header rows and columns remain visible
        on the screen when scrolling.
        """
        
        if p is None:
            return ""

        style = f"""
    <style scoped>
        .dataframe-div {{
          overflow: auto;
          position: relative;
        }}
        .dataframe thead {{
          position: -webkit-sticky; /* for Safari */
          position: sticky;
          top: 0;
          background: #eee;
          color: black;
        }}
        .dataframe thead th:first-child {{
          left: 0;
          z-index: 1;
        }}
        .dataframe tbody tr th {{
          position: -webkit-sticky; /* for Safari */
          position: sticky;
          left: 0;
          background: #eee;
          color: black;
          vertical-align: top;
        }}
    </style>
    """

        return HTML(
            style +
            f'<div class="dataframe-div" style="max-height:{height}px">' 
            + p.dropna().applymap('{:,.2f}'.format).replace('nan', '-').to_html() 
            + "\n</div>"
        )
    
    def disp_ledger(self, df, height=300):
        """
        Show a ledger DataFrame as in KMyMoney.
        This only preserves a subset of the columns
        """
        kmmLedgerCols = ['accountName', 'date', 'payee', 'category', 'reconcile',
                         'quantity', 'price',  # both for Stocks
                         'paiement', 'deposit', 
                         'balance', 'balanceEUR']
        cols = [c for c in kmmLedgerCols if c in df.columns]
        return self.disp(df[cols], height=height)
    
    def _query_detailed_splits(
        self, 
        accounts=None, 
        currency="EUR",
        maxdate=None,     # "1900-01-01"
    ):
        """
        A query that returns data similar to kmmSplits, but each split has
        detailed compatible with both checkins transactions and investment
        transactions:
           `quantity`: the number of units exchanged, in the account's
              currency. For a checking account, this will be EUR for instance,
              and for a stock account it will be the number of shares.
           `price`: the price of each unit for this transaction. For a
              checking account, it will be "1" in general. For a stock account
              it will be the price per share, or null in case of
              "add/remove shares".
           `value`: the value of the transaction (typically quantity*price),
              which does not include any fee paid to a third party.
           `fees`: extra amount paid for banking fees. This is only set for
              stock accounts, and null otherwise.
           `computedPrice`: the price of the unit as of the transaction. This
              is either the price actually used in the transaction, or a price
              coming from the price history.
           `balanceShares`: the current amount of units in the account (i.e.
              the money in the account for a checking account, or the number of
              shares for a stock account)
        """
        test_max_date = "" if maxdate is None else " AND s.postDate <= :maxdate"
        return f"""
        WITH RECURSIVE
        {self._splits_and_fees(currency)},
        {self._price_history(currency)},
        {self._qaccount_name()}
        SELECT 
           kmmAccounts.id as accountId,
           kmmAccounts.currencyId as currencyId,
           s.postDate as date,
           s.transactionId,
           s.splitId,
           s.quantity,
           s.price,
           s.value,
           s.fees, 
           kmmSecurities.id as securityId,

           --  price is either from the transaction, or from the historical prices.
           --  If the account is already in the proper currency, price defaults to 1
           COALESCE(s.price, price_history.computedPrice,
                    CASE kmmAccounts.currencyId
                       WHEN "{currency}" THEN 1
                       ELSE NULL
                    END
           ) as computedPrice,

           --  compute a running total of shares, per account
           SUM(s.quantity) OVER (PARTITION BY s.accountid ORDER BY s.postDate 
                                 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                                ) as balanceShares 
        FROM splits_and_fees s
          JOIN kmmAccounts ON (kmmAccounts.id = s.accountId)
          LEFT JOIN kmmSecurities ON (kmmSecurities.id = kmmAccounts.currencyId)
          LEFT JOIN price_history ON 
             (kmmSecurities.id = price_history.fromId
              AND s.postDate >= price_history.priceDate
              AND s.postDate < price_history.maxDate)
        WHERE TRUE{self._test_accounts('s', accounts)}{test_max_date}
        ORDER BY postDate
            """
    
    def ledger(
        self, 
        accounts=None, 
        currency="EUR",
        mindate=None,     # "1900-01-01"
        maxdate=None,     # "1900-01-01"
    ):
        """
        Compute the list of transactions in a given account, with an output similar
        to kMyMoney. It reports data both in the account's currency (EUR for
        checking accounts for instance, or number of shares for a stock), and in
        EUR, using the historical prices to value the position at the time.
        """
        test_min_date = "" if mindate is None else " AND s.date >= :mindate"
        q = kmm._query_detailed_splits(
            accounts=accounts, currency=currency, maxdate=maxdate)
        return pd.read_sql(
            f"""
            WITH RECURSIVE {self._qaccount_name()}
            SELECT
               qAccountName.name as accountName,
               s.date,
               coalesce(payee.name, '') as payee,  
               destAccount.accountName as category,
               (CASE kmmSplits.reconcileFlag WHEN '2' THEN 'R' WHEN '1' THEN 'C' ELSE '' END) as reconcile,
               (CASE WHEN destS.value > 0 THEN {self._to_float('destS.value')} ELSE NULL END) as paiement,
               (CASE WHEN destS.value <= 0 THEN -{self._to_float('destS.value')} ELSE NULL END) as deposit,
               s.quantity,
               s.price,
               s.balanceShares as balance,
               s.balanceShares * s.computedPrice as balanceEUR
            FROM ({q}) s
               JOIN qAccountName using (accountId)
               JOIN kmmSplits using (transactionId, splitId)
               LEFT JOIN kmmSplits destS on
                  (s.transactionId = destS.transactionId
                   AND s.splitId != destS.splitId)
               LEFT JOIN kmmAccounts destAccount on (destS.accountId = destAccount.id)
               LEFT JOIN kmmPayees payee on (kmmSplits.payeeId = payee.id)
            WHERE TRUE{test_min_date}
            """,
            kmm.sqlite,
            params={
                "mindate": mindate,
            }
        )
    
    def plot_by_category(
        self,
        accounts=None,
        currency=DEFAULT_CURRENCY,
        mindate=None,
        maxdate=None,
        values=['paiement'],  # could include 'deposit'
        expenses=True,   # or Income
        kind="pie",
        subplots=True,       # True if each entry in `values` should be a subplot
    ):
        """
        Generate a plot of paiements and deposits by category, for the given time range.
        Those are not the same as Expense and Income, because nothing prevents us from
        doing a deposit in an Expense account, for instance (e.g. a reimbursement for
        some earlier expense)
        """        
        test_min_date = "" if mindate is None else " AND s.date >= :mindate"
        q = kmm._query_detailed_splits(
            accounts=accounts, currency=currency, maxdate=maxdate)
        p = pd.read_sql(
            f"""
            SELECT
               destAccount.accountName as category,
               destAccount.accountType as categorytype,
               (CASE WHEN destS.value > 0 THEN {self._to_float('destS.value')} ELSE NULL END) as paiement,
               (CASE WHEN destS.value <= 0 THEN -{self._to_float('destS.value')} ELSE NULL END) as deposit,               
               {self._to_float('destS.value')} as amount
            FROM ({q}) s
               JOIN kmmSplits destS on
                  (s.transactionId = destS.transactionId
                   AND s.splitId != destS.splitId)
               JOIN kmmAccounts destAccount on (destS.accountId = destAccount.id)
            WHERE TRUE{test_min_date}
               AND destAccount.accountType IN (:income, :expense)
            """,
            kmm.sqlite,
            params={
                "mindate": mindate,
                "income": ACCOUNT_TYPE.INCOME,
                "expense": ACCOUNT_TYPE.EXPENSE,
            }
        )
        
        if not p.empty:
            p = p[['category'] + list(values)].groupby(['category']).sum()  # group by category
            
            #if kind == 'pie':
            #    p['percent'] = p[value] / p[value].sum() * 100.0
            #    value = 'percent'
            
            p = p.sort_values(values)
            params = dict(
                y=values,
                kind=kind, 
                title="{} - {}".format(mindate or "", maxdate or ""), 
                subplots=subplots,
                legend=None,
                figsize=(20, 10),
                logy=False,
            )
            if kind == 'pie':
                params['autopct'] = '%.2f%%'
            if kind == 'bar' and 'paiement' in p.columns:
                p['paiement'] = -p['paiement']
                
            pl = p.plot(**params)
            
kmm = KMyMoney('/Users/briot/Comptes.kmm')

In [ ]:
# Setup logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
# Direct look at the database (needs `pip install ipython-sql`)
%load_ext sql
sqlite = kmm.sqlite
%sql $sqlite

In [ ]:
# Setup matplotlib
%matplotlib inline

In [ ]:
# List of accounts
bitcoin = 'A000242'      # to check fees
adacore = 'A000061'      # to check values (made up from price)
bourscommun = 'A000106'
eurokidA = 'A000195'
pilotee = 'A000216'
ethereum = 'A000241'

Display the ledger for one or more accounts
---------
A single SQL query is used to compute the lines of the ledger in a format similar to what KMyMoney outputs. We compute the running balance directly in the sql query, which is useful to find operations that brought the balance over some threshold for instance.

We then manipulate the result via Python's Pandas to extract specific information. We cannot, for instance, reduce the range of dates directly in the SQL query, since the running balance is computed on the result rows, so it would be wrong if we remove some rows. (An alternative might be to have a third SELECT statement to filter, but doing that in Pandas is more flexible since it also limits the number of queries we do to sqlite -- keeping everything in memory works fine for a typical kmymoney file)

In [ ]:
kmm.disp_ledger(kmm.ledger(accounts=[ethereum, bitcoin], mindate="2001-05-01"))

Historical prices
------------

All transactions in a given category
------

In [ ]:
p = kmm.ledger(mindate="2019-01-01")  # Get all transactions
cat = (
    'Interne',
    'reconciliation',
    # 'Opening Balances',
)
p = p[ p['category'].isin(cat) ]              # Only keep transactions from specific categories
p = p.drop(['balance', 'reconcile'], axis=1)  # meaningless columns in this view
kmm.disp_ledger(
   p.loc[ 
       p[['deposit', 'paiement']].max(axis=1)  # create a series with max(deposit,paiement)
       .sort_values(ascending=False).index     # sort it, and retrieves the indexes in the original series
   ]  # list the rows of p using the indexes of the sorted series
   .append(p.sum(numeric_only=True), ignore_index=True), # Add a 'Total' row
   height=200
)

Deposits and Paiements for a specific date range
------------
These plots paiements and deposits, not exactly the same as plotting the Expenses and Income accounts, because it is possible to make either paiements or deposits on either of those (for instance a reimbursement for some paiement you made earlier)

In [ ]:
kmm.plot_by_category(mindate="2020-01-01", values=['paiement', 'deposit'])
kmm.plot_by_category(mindate="2020-01-01", values=['amount'], kind='bar')


Net worth by month
---------------
Networth is computed by looking at the current positions in all accounts (EUR or number of shares) at the end of some periods (monthly, yearly,...), and applying the price of the stocks as of the end of that period. In a ledger, the prices are computed as of the transaction itself.

In [ ]:
p = kmm.networth(mindate="2015-01-01", maxdate="2020-12-31", by_year=True)
display(kmm.disp(p, height=600))

p = kmm.networth(mindate="2020-01-01", maxdate="2020-06-31", by_year=False, with_total=True)
display(kmm.disp(p, height=600))

Group the transactions into bins
-----------------------

See https://medium.com/@soulsinporto/group-data-using-bins-and-categories-with-pandas-836c9c9bbd46

In [ ]:
from typing import List, Union
import plotly.express as px


def bin_and_plot(series: pd.Series) -> pd.Series:
    """
    Group data into bins of specific ranges:  [0, 10), [10, 20), ...
    """
    bins = [(0, 50), (50, 100), (100, 200), (200, 500), (500, 1000), 
            (1000, 3000), (3000, 5000), (5000, 10000), (10000, 100000000)]

    intervals = pd.IntervalIndex.from_tuples(bins, closed="left")
    labels = [f"[{l},{r})" for l, r in bins]
    binned = pd.cut(
        series,
        intervals,
        labels=labels,
        precision=0,
        include_lowest=True
    )

    binned.sort_values(ascending=True, inplace=True)
    # Change the values from categorical to string to be able to plot them
    binned = binned.astype("str")

    # For each element in `series`, binned contains the name of the bin it
    # belongs to.
    
    plot_histogram(
        binned, nbins=len(bins), title='Size of paiements',
        axes_titles=['Paiements', ''])

def plot_histogram(
    data_series: pd.Series,
    nbins: int,
    title: str,
    axes_titles: List[Union[str, None]]
) -> None:
    fig = px.histogram(
        x=data_series,
        nbins=nbins,
        title=title
    )

    fig.update_layout(
        xaxis_title=axes_titles[0],
        yaxis_title=axes_titles[1]
    )

    fig.update_layout(
        uniformtext_minsize=14,
        uniformtext_mode="hide",
        bargap=0.1,
        title_x=0.5
    )

    fig.show()

l = kmm.ledger(mindate='2015-01-01')
l = l[ ~l['paiement'].isnull() ]   # Only keep transactions with a paiement
bin_and_plot(l['paiement'])


Ideas
======

- investment value over time (plot)
- compute mean investment price, and current return (`current_price / mean_price`), or using one of the other usual valuation methods that GNUCash provides
- compute total invested in a given investment, and its current book value
- cleanup the `ledger_investment` function, merge with `ledger`
- networth should compute earliest date from database
- networth should display diff between two columns